# Model Notebook

This is the notebook for model in age gender detector. Here, we will extract the data ,train it and fit it into our model, and will evaluate our results and optimize the model for better results.

## Import the Necessary Libraries

In [5]:
import tensorflow as tf
from cv2 import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam,Adagrad,Adadelta,Adamax,RMSprop


ImportError: Bindings generation error. Submodule name should always start with a parent module name. Parent name: cv2.cv2. Submodule name: cv2

## Extract data from the dataser

If data has not been downloaded yet, download it from [Here](https://www.kaggle.com/jangedoo/utkface-new)

In [ ]:
fldr="./UTkFace"

In [ ]:
import os
flies=os.listdir(fldr)

## Create List of images from the dataset

In [ ]:
ages=[]
genders=[]
images=[]

for fle in flies:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    total=fldr+'/'+fle
    print(total)
    image=cv2.imread(total)

    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(48,48))
    images.append(image)


## Create List of Age and Gender from the dataset

In [ ]:
for fle in flies:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    ages.append(age)
    genders.append(gender)

## Viewing a sample

In [ ]:
plt.imshow(images[87])

In [ ]:
print(ages[87])

In [ ]:
print(genders[87])

## Saving images array,ages array, and genders array in the system

In [ ]:
images_f=np.array(images)
ages_f=np.array(ages)
genders_f=np.array(genders)

In [ ]:
np.save(fldr+'image.npy',images_f)
np.save(fldr+'ages.npy',ages_f)
np.save(fldr+'genders.npy',genders_f)

## Finding the no. of Male and Female samples respectively

In [ ]:
values, counts=np.unique(genders_f,return_counts=True)
print(counts)

## Plotting the No. of Male and Female Samples

In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
gender=['Male','Female']
values=[12391,11317]
ax.bar(gender,values)
plt.show()

## Finding The no. of samples in each age

In [ ]:
values, counts=np.unique(ages_f,return_counts=True)
print(counts)

## Converting Age counts to lists

In [ ]:
val=values.tolist()
cnt=counts.tolist()

## Plotting The Samples Agewise

In [ ]:
plt.plot(counts)
plt.xlabel('Ages')
plt.ylabel('Distribution')
plt.show()

## Definig Labes for Output

In [ ]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    labels.append(label)
    i=i+1

## Creating another list of images from existing one
### For better learning

In [ ]:
images_f_2=images_f/255

In [ ]:
images_f_2.shape

In [ ]:
labels_f=np.array(labels)

## Splitting The Dataset into test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(images_f_2,labels_f,test_size=0.25)

In [ ]:
Y_train[0:5]

## Splitting 'Y_test' and 'Y_train' further
### For better understanding  

In [ ]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:5]

In [ ]:
Y_train_2[1][0:5]

## Defining the Model

In [ ]:
def Convolution(input_tensor,filters):
    x=Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.001))(input_tensor)
    x=Dropout(0.1)(x)
    x=Activation('relu')(x)
    return x


In [ ]:
def model(input_shape):
    inputs=Input((input_shape))
    conv_1=Convolution(inputs,32)
    maxp_1=MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2=Convolution(maxp_1,64)
    maxp_2=MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3=Convolution(maxp_2,128)
    maxp_3=MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4=Convolution(maxp_3,256)
    maxp_4=MaxPooling2D(pool_size=(2,2))(conv_4)
    flatten= Flatten()(maxp_4)
    dense_1=Dense(64,activation='relu')(flatten)
    dense_2=Dense(64,activation='relu')(flatten)
    drop_1=Dropout(0.2)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    output_1=Dense(1,activation='sigmoid',name='sex_out')(drop_1)
    output_2=Dense(1,activation='relu',name='age_out')(drop_2)
    model=Model(inputs=[inputs],outputs=[output_1,output_2])
    model.compile(loss=["binary_crossentropy","mae"],optimizer="Adam",metrics=["accuracy"])
    return model

In [ ]:
Model=model((48,48,3))

In [ ]:
Model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

## Initializing the Model

In [ ]:
fle_s='Age_Sex_Detection.h5'
checkpoint=ModelCheckpoint(fle_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75,monitor='val_loss',restore_best_weights='True')
callback_list=[checkpoint,Early_stop]

## Running the Model
### This will take some time 

In [ ]:
History=Model.fit(X_train,Y_train_2,batch_size=64,validation_data=(X_test,Y_test_2),epochs=250,callbacks=callback_list)

## Model Evaluation

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [ ]:
pred=Model.predict(X_test)

In [ ]:
pred[1]

## Plotting Loss

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)


## Plotting Sex Accuracy

In [ ]:
plt.plot(History.history['sex_out_accuracy'])
plt.plot(History.history['val_sex_out_accuracy'])
plt.title('Model Accuracy')
plt.xlabel=('Epoch')
plt.ylabel=('Accuracy')
plt.legend(['Train','Validation'],loc='upper left')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

## Plotting Gender Accuracy

In [ ]:
fig,ax=plt.subplots()
ax.scatter(Y_test_2[1],pred[1])
ax.plot([Y_test_2[1].min(),Y_test_2[1].max()],[Y_test_2[1].min(),Y_test_2[1].max()],'k--',lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

## Making Report of the Model (Gender Only)

In [ ]:
i=0
Pred_l=[]
while(i<len(pred[0])):
    Pred_l.append(int(np.round(pred[0][i])))
    i=i+1
    

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [ ]:
report=classification_report(Y_test_2[0],Pred_l)

In [ ]:
print(report)

## Making Heatmap of the Model (Gender only)

In [ ]:
results=confusion_matrix(Y_test_2[0],Pred_l)

In [ ]:
import seaborn as sns
sns.heatmap(results,annot=True)

## Testing The Images Ourself

In [ ]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    pred_l=Model.predict(np.array([image_test]))
    sex_f=['Male','Female']
    age=int(np.round(pred_l[1][0]))
    sex=int(np.round(pred_l[0][0]))
    print("Predicted Age is "+ str(age))
    print("Predicted Gender is "+ sex_f[sex])
    

### Test 1

In [ ]:
test_image(4,images_f,images_f_2,Model)

### Test 2

In [ ]:
test_image(23,images_f,images_f_2,Model)

### Testt 3

In [ ]:
test_image(168,images_f,images_f_2,Model)

### Test 4
Error in age in this case

In [ ]:
test_image(888,images_f,images_f_2,Model)

### Test 5

In [ ]:
test_image(1288,images_f,images_f_2,Model)

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
def load_and_preprocess_data(csv_path, img_dir):
    data = pd.read_csv(csv_path)
    images = []
    ages = []
    
    for index, row in data.iterrows():
        img_path = os.path.join(img_dir, row['file_name'])
        age = row['age']
        try:
            img = Image.open(img_path)
            img = img.resize((48, 48))
            img = np.array(img)
            if img.shape == (48, 48, 3):  # Ensure image has 3 channels
                images.append(img)
                ages.append(age)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    
    images = np.array(images)
    ages = np.array(ages)
    
    # Normalize images
    images = images / 255.0
    
    return images, ages

# Paths to the dataset files
csv_path = 'path/to/your/dataset.csv'  # Replace with the path to your CSV file
img_dir = 'path/to/your/images'  # Replace with the path to your image directory

# Load data
images, ages = load_and_preprocess_data(csv_path, img_dir)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)



FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/dataset.csv'

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [7]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [8]:
# Load and preprocess the dataset
def load_and_preprocess_data(csv_path, img_dir):
    data = pd.read_csv(csv_path)
    images = []
    ages = []
    
    for index, row in data.iterrows():
        img_path = os.path.join(img_dir, row['file_name'])
        age = row['age']
        try:
            img = Image.open(img_path)
            img = img.resize((48, 48))
            img = np.array(img)
            if img.shape == (48, 48, 3):  # Ensure image has 3 channels
                images.append(img)
                ages.append(age)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    
    images = np.array(images)
    ages = np.array(ages)
    
    # Normalize images
    images = images / 255.0
    
    return images, ages

# Paths to the dataset files
csv_path = 'path/to/your/dataset.csv'  # Replace with the path to your CSV file
img_dir = 'path/to/your/images'  # Replace with the path to your image directory

# Load data
images, ages = load_and_preprocess_data(csv_path, img_dir)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/dataset.csv'